# Tensores con Pytorch

Uno de los conceptos más frecuentes en machine learning es el de **tensor**.   
Un tensor es la generalización del concepto de vector utilizando el rango para definir el número de índices del tensor. Por ejemplo, un vector es un tensor de rango 1, una matriz es un tensor de rango 2 y un tensor de rango tres sería una matriz con un eje extra.    
La clase **torch.tensor** de PyTorch pemite trabajar cómodamente con tensores, tanto en la CPU como en la GPU. Existen diferentes formas de crear un tensor.

In [3]:
import torch
import numpy as np

#A partir de una lista de Python
list = ([1,2],[3,4],[5,6])
tensor1=torch.tensor(list)
print("Tensor 1:")
print(tensor1)
print()

#A partir de los datos directamente
tensor2=torch.tensor([[[1, 2, 3],[4, 5, 6]],[[1, 2, 3],[4, 5, 6]]])
print("Tensor 2:")
print(tensor2)
print()

#A partir de un array numpy
tensor3=torch.tensor(np.array([1,2,3,4]))
print("Tensor 3:")
print(tensor3)
print()

#Usando una operación de creación de tensores
tensor4=torch.rand(2,3)
print("Tensor 4:")
print(tensor4)

Tensor 1:
tensor([[1, 2],
        [3, 4],
        [5, 6]])

Tensor 2:
tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]]])

Tensor 3:
tensor([1, 2, 3, 4])

Tensor 4:
tensor([[0.7711, 0.5063, 0.8099],
        [0.6221, 0.2899, 0.0663]])


In [4]:
#A través del método size se obtiene el tamaño del tensor, su rango y las dimensiones

#Un tensor de rango 2 (3,2) para tensor1
print(tensor1.size())

#Un tensor de rango 3 con las dimensiones (2,2,3) para tensor2
print(tensor2.size())

torch.Size([3, 2])
torch.Size([2, 2, 3])


In [6]:
#A los componentes de los tensores se puede acceder como usualmente se efectua en Python
print(tensor1[0])
print(tensor1[0,1])

#Con view se cambia las dimensiones y el rango del tensor
tensor5=torch.rand(2,4)
print(tensor5.size())

tensor6=tensor5.view(8)
print(tensor6.size())

tensor([1, 2])
tensor(2)
torch.Size([2, 4])
torch.Size([8])


In [7]:
#Un tensor se puede crear con requires_grad=True para que torch.autograd registre las operaciones y realice la diferenciación automática

x = torch.tensor([2],dtype=torch.float,requires_grad=True)
y = x.pow(3)
y.backward()
x.grad

tensor([12.])

In [8]:
#PyTorch permite trabajar con CUDA para realizar las operaciones en la GPU
torch.cuda.is_available()
torch.cuda.current_device()
print(torch.cuda.get_device_name(0))
# Se produce un error si no se dispone de GPU instalada y configurada en el sistema.

AssertionError: Torch not compiled with CUDA enabled

In [9]:
#Comparación del tiempo en realizar una multiplicaión de matrices en la CPU y en la GPU
%%time
for i in range(500):
  x = torch.randn(1000,1000)
  y = torch.randn(1000,1000)
  z = torch.matmul(x,y)

UsageError: Line magic function `%%time` not found.


In [10]:
#Se realiza la multiplicación en la GPU
%%time
cuda0 = torch.device('cuda:0')
for i in range(500):
  x = torch.randn(1000,1000,device = cuda0)
  y = torch.randn(1000,1000,device =cuda0)
  z = torch.matmul(x,y)

UsageError: Line magic function `%%time` not found.


## Almacenamiento de tensores

Como hemos visto, los tensores son un elemento básico en PyTorch. Son los elementos de diferente rango (escalares, vectores, matrices, etc.) que contienen la información en PyTorch y sobre los que actúan los modelos.

Sin embargo, a pesar de su rango y dimensiones, los tensores se almacenan en memoria en un array unidimensional de elementos contiguos. De esta forma se homogeiniza su almacenamiento y se hace más eficiente su tratamient

A esta información se accede a través de la clase Torch.storage(). A continuación, vemos un ejemplo de un tensor y su almacenamiento en un array unidimensional.


In [11]:
import torch 

tensor1 = torch.tensor([[2.0, 1.0], [4.0, 3.0], [2.0, 1.0]])
tensor1.storage()


C:\Users\Jordi\AppData\Local\Temp\ipykernel_5632\3283328093.py:4: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  tensor1.storage()


 2.0
 1.0
 4.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

Los tensores son vistas de este almacenamiento y para poder relacionar las diferentes vistas con un único almacenamiento, PyTorch hace uso de los metadatos size, offset y stride.

- *Size* es como shape en numpy e indica el número de elementos en cada dimensión. 
- *Offset* es el índice en el almacenamiento (storage) que indica donde está el primer elemento del tensor.
- El *stride* es el número de desplazamientos en el almacenamiento necesarios para indexar el siguiente elemento en cada dimensión.

A continuación, vemos estos metadatos para nuestro tensor.

In [12]:
print("El tamaño (size) es", tensor1.size())

print("El offset es", tensor1.storage_offset())

print("El stride es", tensor1.stride())

El tamaño (size) es torch.Size([3, 2])
El offset es 0
El stride es (2, 1)


Como se puede ver, nuestro tensor tiene un tamaño de 3 filas y dos columnas y el primer elemento del tensor está en la posición cero del almacenamiento.

Para movernos una posición en la primera dimensión/segunda dimensión (fila/columna) necesitamos avanzar (2,1) posiciones en el almacenamiento.

A continuación creamos otro tensor de 3 filas y 3 columnas y vemos como ahora para movernos en la primera dimensión (fila) necesitamos avanzar 3 posiciones en el almacenamiento.

In [13]:
tensor2=torch.rand(3,3)

print("El tamaño (size) es", tensor2.size())

print("El offset es", tensor2.storage_offset())

print("El stride es", tensor2.stride())

El tamaño (size) es torch.Size([3, 3])
El offset es 0
El stride es (3, 1)


Un tensor cuya vista coincida con el almacenamiento es un **tensor contiguo**. Es decir, sus valores se almacenan desde la dimensión más a la derecha hacia adelante. Esto es conveniente en términos de eficiencia porque podemos recorrer el tensor en orden sin hacer saltos en la memoria.

Hay una serie de operaciones en PyTorch que no cambian el almacenamiento pero sí que cambian la vista y, en consecuencia, los metadatos de vista del tensor. Son las operaciones *narrow(), view(), expand()* y *transpose()*.

Vamos a transponer nuestro primer vector y ver su storage y sus metadatos:

In [14]:
tensor1_transpose=tensor1.transpose(0,1)
print(tensor1_transpose)
print(tensor1_transpose.storage())

print("¿Es el tensor contiguo?", tensor1_transpose.is_contiguous())

print("El tamaño (size) es", tensor1_transpose.size())

print("El offset es", tensor1_transpose.storage_offset())

print("El stride es", tensor1_transpose.stride())

tensor([[2., 4., 2.],
        [1., 3., 1.]])
 2.0
 1.0
 4.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
¿Es el tensor contiguo? False
El tamaño (size) es torch.Size([2, 3])
El offset es 0
El stride es (1, 2)


Como se puede observar, el almacenamiento interno del tensor transpuesto con dos filas y tres columnas no cambia pero sí que cambian los metadatos de vista. Ahora el tensor no es contiguo y el stride es (1,2), por lo que necesitamos avanzar (1,2) posiciones en el almacenamiento para avanzar una posición en la (fila/columna).